# Bokeh Tutorial #3: Layouts, Widgets, and Interactions

In [1]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

Layouts are very important for linking plots together for interactions. In order to be able to create widgets that access multiple plots at once, you have to be able to create a document containing multiple elements. One way to accomplish this is by using **gridplot**

In [3]:
from bokeh.layouts import gridplot

x = list(range(11))
y0, y1, y2 = x, [10-i for i in x], [abs(i-5) for i in x]

#create a new plot
s1 = figure(width=250, plot_height=250)
s1.circle(x, y0, size=10, color='#3B547A')

#create another plot
s2 = figure(width=250, plot_height=250)
s2.triangle(x, y1, size=10, color='#f35800')

#one more
s3 = figure(width=250, plot_height=250)
s3.cross(x, y2, size=10, color='#474544')

#put them all together with gridplot
p = gridplot([[s1, s2, s3]], toolbar_location=None)

show(p)

*Exercise: Make your own gridplot in bokeh*

In [4]:
from bokeh.sampledata.iris import flowers as df

df.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

In [9]:
from bokeh.models import ColumnDataSource

source = ColumnDataSource(df)

In [10]:
from bokeh.palettes import Viridis8 as colors
colors

['#440154',
 '#46317E',
 '#365A8C',
 '#277E8E',
 '#1EA087',
 '#49C16D',
 '#9DD93A',
 '#FDE724']

In [19]:
rows_of_plots = []
i = 0

for x_data in df.columns[:4]:
    single_row = []
    color=colors[2*i]
    
    for y_data in df.columns[:4]:
        plot = figure(width=150, plot_height=150)
        plot.circle(x_data, y_data, size=10, color=color, source=source)
        single_row.append(plot)

    rows_of_plots.append(single_row)  
    i += 1

In [20]:
p = gridplot(rows_of_plots, toolbar_location=None)
show(p)

You can also arrange plots horizontally and vertically with the *row()* and *column()* functions in **bokeh.layouts**.

*Exercise: Try a vertical or horizontal for a group of plots*

In [40]:
from bokeh.layouts import column
from bokeh.sampledata.autompg import autompg as df
from bokeh.palettes import Dark2 as dark_colors

plots = []
i = 0

for feature in df.columns[1:4]:
    plot = figure(width=500, plot_height=500)
    plot.square(x=df[feature], y=df[df.columns[0]], color=dark_colors[i+3][i], size=3)
    plots.append(plot)
    i += 1
    
p = column(plots, name='Auto MPG Data')
show(p)

## Linked Interactions

Once plots are grouped, it's possible to link interactions to multiple plots, So, actions like panning one plot with update all of the plots, and selecting data will update to select the same data points on other plots.

### Linked Panning

Linked panning is very simple in Bokeh. Just share the appropriate range objects between the plots you want to link.

In [41]:
plot_options = dict(width=250, plot_height=250, tools='pan,wheel_zoom')

# create a new plot
s1 = figure(**plot_options)
s1.circle(x, y0, size=10, color="navy")

# create a new plot and share both ranges
s2 = figure(x_range=s1.x_range, y_range=s1.y_range, **plot_options)
s2.triangle(x, y1, size=10, color="firebrick")

# create a new plot and share only one range
s3 = figure(x_range=s1.x_range, **plot_options)
s3.square(x, y2, size=10, color="olive")

p = gridplot([[s1, s2, s3]])

# show the results
show(p)

*Exercise: create two plots in a gridplot and link their ranges.*

In [49]:
plot_options = dict(width=350, height=350, tools='pan,wheel_zoom')

source = ColumnDataSource(df)

s1 = figure(**plot_options)
s1.cross('weight', 'mpg', size=7, color=dark_colors[5][2], source=source)

s2 = figure(y_range=s1.y_range, **plot_options)
s2.diamond('hp', 'mpg',size=7, color=dark_colors[5][4], source=source)

p = gridplot([[s1, s2]])
show(p)

### Linked Brushing

Linked selection is similarly done, by making sure plots share a data source. However, data sources must be passed explicitly in this case using ColumnDataSource

In [50]:
x = list(range(-20, 21))
y0, y1 = [abs(xx) for xx in x], [xx**2 for xx in x]

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS = "box_select,lasso_select,help"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=300, height=300)
left.circle('x', 'y0', source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=300, height=300)
right.circle('x', 'y1', source=source)

p = gridplot([[left, right]])

show(p)

*Exercise: create two plots in a gridplot and link the data sources*

In [53]:
tools = 'pan,wheel_zoom,box_select,lasso_select,help,reset'

plot_options = dict(width=350, height=350, tools=tools)

source = ColumnDataSource(df)

s1 = figure(**plot_options)
s1.diamond('weight', 'mpg', size=7, color=dark_colors[5][2], source=source)

s2 = figure(y_range=s1.y_range, **plot_options)
s2.diamond('hp', 'mpg',size=7, color=dark_colors[5][4], source=source)

p = gridplot([[s1, s2]])
show(p)

## Hover Tools

Bokeh's hover tool allows for a tooltip to show details about the data points that the mouse is hovered over. Basic configuration involves passing a list of tuples into the HoverTool() function for each data field.

In [57]:
from bokeh.models import HoverTool

source = ColumnDataSource(
        data=dict(
            x=[1, 2, 3, 4, 5],
            y=[2, 5, 8, 2, 7],
            desc=['A', 'b', 'C', 'd', 'E'],
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
            ("desc", "@desc"),
        ]
    )

p = figure(plot_width=500, plot_height=500, tools=[hover], title="Mouse over the dots")

p.circle('x', 'y', size=20, source=source)
show(p)

In [59]:
import numpy as np
from bokeh.io import push_notebook

x = np.linspace(0, 2*np.pi, 2000)
y = np.sin(x)

source = ColumnDataSource(data=dict(x=x, y=y))

p = figure(title="simple line example", plot_height=300, plot_width=600, y_range=(-5, 5))
p.line(x, y, color="#2222aa", alpha=0.5, line_width=2, source=source, name="foo")

def update(f, w=1, A=1, phi=0):
    if   f == "sin": func = np.sin
    elif f == "cos": func = np.cos
    elif f == "tan": func = np.tan
    source.data['y'] = A * func(w * x + phi)
    push_notebook()
    
show(p, notebook_handle=True)

C:\Users\Tyler Hughes\AppData\Local\Continuum\Anaconda3\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
C:\Users\Tyler Hughes\AppData\Local\Continuum\Anaconda3\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


In [61]:
from ipywidgets import interact
interact(update, f=["sin","cos","tan"], w=(0,10,0.1), A=(0,5,0.1), phi=(0,10,0.1))

## Widgets

Bokeh provides a small set of basic widgets and relies on CustomJS callbacks written by the user for functionality beyond what it provides.

Using widgets is very similar to laying out a plot.

In [63]:
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Slider

slider = Slider(start=0, end=10, value=1, step=.1, title='foo')
show(widgetbox(slider))

*Exercise: Create and show a Select widget*

In [64]:
from bokeh.models.widgets import Select

help(Select)

Help on class Select in module bokeh.models.widgets.inputs:

class Select(InputWidget)
 |  Single-select widget.
 |  
 |  Method resolution order:
 |      Select
 |      InputWidget
 |      bokeh.models.widgets.widget.Widget
 |      bokeh.models.layouts.LayoutDOM
 |      bokeh.model.Model
 |      bokeh.core.has_props.HasProps
 |      bokeh.util.callback_manager.CallbackManager
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  get_class(view_model_name) from builtins.type
 |      Given a __view_model__ name, returns the corresponding class
 |      object
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  callback
 |      A callback to run in the browser whenever the current Select dropdown
 |      value changes.
 |  
 |  options
 |      Available selection options. Options may be provided either as a list of
 |      possible string values, or as a list of tuples, each of the form
 |      ``(value, lab

In [65]:
select = Select(options=['Yes','No', 'Maybe'], title='Do you like me?')

show(widgetbox(select))

## Custom JS Callbacks

In [67]:
from bokeh.models import TapTool, CustomJS, ColumnDataSource

callback = CustomJS(code="alert('hello world')")
tap = TapTool(callback=callback)

p = figure(plot_width=600, plot_height=300, tools=[tap], title="Try clicking a data point!")

p.circle('x', 'y', size=20, source=ColumnDataSource(data=dict(x=[1, 2, 3, 4, 5], y=[2, 5, 8, 2, 7])))

show(p)

As evidenced above, one can attach JavaScript actions to tools and widgets in order to customize the visualizations interactivity. Execution of the callback occurs when the value of the widget changes. To map names to Python bokeh ColumnDataSources, the CustomJS object accepts a Python dictionary object as a list of "arguments".

In [68]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider

x = [x*0.005 for x in range(0, 200)]
y = x

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source), code="""
    var data = source.get('data');
    var f = cb_obj.get('value')
    x = data['x']
    y = data['y']
    for (i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    source.trigger('change');
""")

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power", callback=callback)

layout = column(slider, plot)

show(layout)

### Callbacks for Selections

One can have a CustomJS callback trigger when a user selection changes as well.

In [69]:
from random import random

x = [random() for x in range(500)]
y = [random() for y in range(500)]
color = ["navy"] * len(x)

s = ColumnDataSource(data=dict(x=x, y=y, color=color))
p = figure(plot_width=400, plot_height=400, tools="lasso_select", title="Select Here")
p.circle('x', 'y', color='color', size=8, source=s, alpha=0.4)

s2 = ColumnDataSource(data=dict(ym=[0.5, 0.5]))
p.line(x=[0,1], y='ym', color="orange", line_width=5, alpha=0.6, source=s2)

s.callback = CustomJS(args=dict(s2=s2), code="""
    var inds = cb_obj.get('selected')['1d'].indices;
    var d = cb_obj.get('data');
    var ym = 0
    
    if (inds.length == 0) { return; }
    
    for (i = 0; i < d['color'].length; i++) {
        d['color'][i] = "navy"
    }
    for (i = 0; i < inds.length; i++) {
        d['color'][inds[i]] = "firebrick"
        ym += d['y'][inds[i]]
    }
    
    ym /= inds.length
    s2.get('data')['ym'] = [ym, ym]
    
    cb_obj.trigger('change');
    s2.trigger('change');
""")

show(p)

C:\Users\Tyler Hughes\AppData\Local\Continuum\Anaconda3\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
